<a href="https://colab.research.google.com/github/kimmikaka036/TCB-Python/blob/master/Temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -o '/content/drive/My Drive/A-Dataset/dataset-crop.zip' 

In [0]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.io import imread

def read_image2(filename ,byteorder='>'):
    img = load_img(filename)  # this is a PIL image
    img.thumbnail((img.width, img.height))
    # Convert to Numpy Array
    x = img_to_array(img)   
    return x
  
def read_image3(filename, byteorder='>'):
    
    #first we read the image, as a raw file to the buffer
    im = imread(filename, as_grey = 1)
    images = np.asarray(im, dtype=np.float32)
    images = images / 255.0 
    images = images.reshape( 224, 224) 
    return  im 
  
def read_image4(filename ,byteorder='>'):
    with open(filename, 'rb') as f:
        buffer = f.read() 
        
    return np.frombuffer(buffer,
                         dtype=np.float32,
                         count=224*224, 
                         ).reshape(224, 224)
  

image = read_image3('/content/dataset-crop/s1/1.png', 'rw+')
image.shape

/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:48: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


(224, 224)

In [0]:
size = 2
total_sample_size = 1000


def get_data(size, total_sample_size):
    #read the image
    image = read_image3('/content/dataset-crop/s' + str(1) + '/' + str(1) + '.png', 'rw+')
    #reduce the size
    image = image[::size, ::size]
    #get the new size
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0
    
    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2]) # 2 is for pairs
    y_genuine = np.zeros([total_sample_size, 1])
    
    for i in range(100):
        for j in range(int(total_sample_size/100)):
            ind1 = 0
            ind2 = 0
            
            #read images from same directory (genuine pair)
            while ind1 == ind2:
                ind1 = np.random.randint(2)
                ind2 = np.random.randint(2)
            
            # read the two images
            img1 = read_image3('/content/dataset-crop/s' + str(i+1) + '/' + str(ind1 + 1) + '.png', 'rw+')
            img2 = read_image3('/content/dataset-crop/s' + str(i+1) + '/' + str(ind2 + 1) + '.png', 'rw+')
            
            #reduce the size
            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]
            
            #store the images to the initialized numpy array
            x_geuine_pair[count, 0, 0, :, :] = img1
            x_geuine_pair[count, 1, 0, :, :] = img2
            
            #as we are drawing images from the same directory we assign label as 1. (genuine pair)
            y_genuine[count] = 1 
            count += 1

    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])
    y_imposite = np.zeros([total_sample_size, 1])
    
    for i in range(int(total_sample_size/2)):
        for j in range(2):
            
            #read images from different directory (imposite pair)
            while True:
                ind1 = np.random.randint(100)
                ind2 = np.random.randint(100)
                if ind1 != ind2:
                    break
                    
            img1 = read_image3('/content/dataset-crop/s' + str(ind1+1) + '/' + str(j + 1) + '.png', 'rw+')
            img2 = read_image3('/content/dataset-crop/s' + str(ind2+1) + '/' + str(j + 1) + '.png', 'rw+')

            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]

            x_imposite_pair[count, 0, 0, :, :] = img1
            x_imposite_pair[count, 1, 0, :, :] = img2
            #as we are drawing images from the different directory we assign label as 0. (imposite pair)
            y_imposite[count] = 0
            count += 1
            
    #now, concatenate, genuine pairs and imposite pair to get the whole data
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y


In [0]:
X, Y = get_data(size, total_sample_size) 
print(X.shape)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)
print(x_train.shape)
# print(X)


/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:48: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


(2000, 2, 1, 112, 112)
(1500, 2, 1, 112, 112)


In [0]:
def build_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x) 

In [0]:
input_dim = x_train.shape[2:]
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
feat_vecs_a = base_network(img_a)
feat_vecs_b = base_network(img_b)

In [0]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [0]:
epochs = 13


model = Model(input=[img_a, img_b], output=distance)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`
  after removing the cwd from sys.path.


In [0]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))
  
def accuracy(y_true, y_pred): 
      return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))
  
rms = RMSprop(lr =0.001)

model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

In [0]:

img_1 = x_train[:, 0]
img_2 = x_train[:, 1] 
 
img_3 = x_test[:, 0]
img_4 = x_test[:, 1] 

# model.fit([img_1, img_2], y_train,
#           batch_size=128,
#           epochs=epochs,
#           validation_split=.25)

model.fit([img_1, img_2], y_train,
          batch_size=128,
          epochs=epochs,
          validation_data=([img_3, img_4], y_test))

# 

Train on 1500 samples, validate on 500 samples
Epoch 1/13
1500/1500 [==============================] - 1s 886us/step - loss: 0.1300 - accuracy: 0.8247 - val_loss: 0.1092 - val_accuracy: 0.8600
Epoch 2/13
1500/1500 [==============================] - 1s 491us/step - loss: 0.1307 - accuracy: 0.8167 - val_loss: 0.1066 - val_accuracy: 0.8540
Epoch 3/13
1500/1500 [==============================] - 1s 467us/step - loss: 0.1271 - accuracy: 0.8267 - val_loss: 0.1088 - val_accuracy: 0.8480
Epoch 4/13
1500/1500 [==============================] - 1s 460us/step - loss: 0.1330 - accuracy: 0.8160 - val_loss: 0.1071 - val_accuracy: 0.8540
Epoch 5/13
1500/1500 [==============================] - 1s 461us/step - loss: 0.1219 - accuracy: 0.8340 - val_loss: 0.1034 - val_accuracy: 0.8620
Epoch 6/13
1500/1500 [==============================] - 1s 464us/step - loss: 0.1225 - accuracy: 0.8260 - val_loss: 0.1065 - val_accuracy: 0.8580
Epoch 7/13
1500/1500 [==============================] - 1s 469us/step - loss:

In [0]:
def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

In [0]:
pred = model.predict([x_test[:, 0], x_test[:, 1]])
compute_accuracy(pred, y_test)

0.8143322475570033